In [ ]:
import networkx as nx
import numpy as np

import pystac_client
from oxeo.water.models.segmentation import Segmentation2DPredictor
from oxeo.core.models.tile import load_tile_from_stac_as_dict, load_aoi_from_stac_as_dict, tile_from_id, TilePath, tile_to_geom
from oxeo.core.models import data
URL = "https://earth-search.aws.element84.com/v0"
catalog = pystac_client.Client.open(URL)
import matplotlib.pyplot as plt

In [ ]:
predictor = Segmentation2DPredictor(ckpt_path="../data/semseg_epoch_012.ckpt", fs=None,
                                    bands=["B08", "B04", "B03", "B02", "B11", "B12"])

In [ ]:
tile_id = "35_K_10000_56_758"
tile = tile_from_id(tile_id)
tile_path = TilePath(tile, "sentinel-2", URL)

In [ ]:

search_params = {
                 "bbox":tile_path.tile.bbox_wgs84, #min_x, min_y, max_x, max_y
                 "collections":["sentinel-s2-l2a-cogs"],
                 #"datetime":"2020-04-01/2020-05-01"
                 }

sample = load_aoi_from_stac_as_dict(URL, search_params,
                          bands=["B08", "B04", "B03", "B02", "B11", "B12"],
                           revisit=slice(0,2), 
                           chunk_aligned=True,median=True)

In [ ]:
plt.imshow(sample["image"][[1,2,3]].transpose(1,2,0)/10000)

In [ ]:
# we can predict a stac aoi
pred = predictor.predict_stac_aoi(catalog_url=URL, 
                                  collections=["sentinel-s2-l2a-cogs"], 
                                  constellation="sentinel-2", 
                                  bbox=tile_path.tile.bbox_wgs84,
                                  revisit=slice(0,2),
                                  chunk_aligned=True)

In [ ]:
plt.imshow(pred[0])

In [ ]:
# We can also predict a tile directly using stac
pred = predictor.predict(tile_path, slice(0,2),
                  use_stac=True, 
                  stac_kwargs= {"catalog_url": URL,
                                "collections":["sentinel-s2-l2a-cogs"]})

In [ ]:
# Let's get an AOI to work with

search_params = {
                 "bbox":tile_path.tile.bbox_wgs84, #min_x, min_y, max_x, max_y
                 "collections":["sentinel-s2-l2a-cogs"],
                 "datetime":"2020-04-01/2020-05-01"
                 }
aoi = data.get_aoi_from_stac_catalog(catalog_url=URL, 
                                     search_params=search_params,
                                     chunk_aligned=True)
coords = aoi.isel(time=0).sel(band=["B04", "B03", "B02"]).coords


In [ ]:
from oxeo.core.models import graph 

In [ ]:
G = nx.Graph()
# Create graph with empty schema
G.add_nodes_from([
    ("tile_1", graph.NODE_SCHEMA),
    ("tile_2", graph.NODE_SCHEMA),
    ("wb_1", {"name": "Victoria"})
])
G.add_edge("tile_1", "wb_1", rel="isin")
G.add_edge("tile_2", "wb_1", rel="isin")

In [ ]:
# Append stackstac xa attrs to graph nodes
coords_dict = graph.stackstac_xa_coords_to_dict(coords)
graph.append_attrs(G, 'tile_1', attrs=coords_dict, appendable_attrs=["time", "res_level"])
graph.append_attrs(G, 'tile_2', attrs=coords_dict, appendable_attrs=["time", "res_level"])

# append res_level attrs to wb_1
graph.append_attrs(G, 'tile_1', attrs={"res_level":2}, appendable_attrs=["time", "res_level"])
graph.append_attrs(G, 'tile_1', attrs={"res_level":10}, appendable_attrs=["time", "res_level"])

# append res_level attrs to wb_2
graph.append_attrs(G, 'tile_2', attrs={"res_level":2}, appendable_attrs=["time", "res_level"])
graph.append_attrs(G, 'tile_2', attrs={"res_level":3}, appendable_attrs=["time", "res_level"])

In [ ]:
# Get the avg 
asset_avg_res_levels = []
for wb in list(G.neighbors("wb_1")):
    asset_avg_res_levels.append(np.mean(G.nodes[wb]["res_level"]))
    
asset_avg_res_levels